In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset,TensorDataset
from autoencoder import Autoencoder
import torchvision
from model2 import classification_model
import copy
import partition
from pca import PCADigitReducer
from autoencoder import reduce_dimensions
from training import train,test, train_fashion,test_fashion
from federated_learning import distribute_global_model, federated_averaging
from model4 import MultilayerPerceptron
import cluster

# Data Preprocessing

In [2]:
# Predefined stuff

n_epochs = 5
batch_size_train = 100
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10
num_clusters = 2

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [3]:
# training and testing data as dataloaders

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [4]:
train_loader_pca = copy.copy(train_loader)
test_loader_pca = copy.copy(test_loader)

train_loader_auto = copy.copy(train_loader)
test_loader_auto = copy.copy(test_loader)

In [5]:
class CustomTensorDataset(TensorDataset):
    def __init__(self, *tensors):
        super().__init__(*tensors)
        self.data = tensors[0]
        self.targets = tensors[1] 

# PCA

In [6]:
train_data = []
train_labels = []
for data, labels in train_loader_pca:
    train_data.append(data.view(data.size(0), -1))  
    train_labels.append(labels)
train_data = torch.cat(train_data, dim=0)  
train_labels = torch.cat(train_labels, dim=0)

train_data_np = train_data.numpy()

pca = PCADigitReducer(100)
train_data_reduced = pca.fit_transform(train_data_np)  

train_data_reconstructed_np = pca.inverse_transform(train_data_reduced) 
train_data_reconstructed = torch.tensor(train_data_reconstructed_np, dtype=torch.float32)

train_data_reconstructed = train_data_reconstructed.view(-1, 1, 28, 28)

train_data_reconstructed = (train_data_reconstructed - 0.1307) / 0.3081

batch_size_train = train_loader_pca.batch_size
train_dataset_pca = CustomTensorDataset(train_data_reconstructed, train_labels)
train_loader_reduced_pca = DataLoader(train_dataset_pca, batch_size=batch_size_train, shuffle=True)

# Autoencoder

In [7]:
latent_dim = 100  
autoencoder = Autoencoder(latent_dim=latent_dim)
auto_criterion = nn.MSELoss()
auto_optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)
auto_num_epochs = 5
for epoch in range(auto_num_epochs): 
    for images, _ in train_loader_auto:
        auto_optimizer.zero_grad()
        reconstructed = autoencoder(images)
        loss = auto_criterion(reconstructed, images)  
        loss.backward()
        auto_optimizer.step()
        
    print(f"Epoch [{epoch+1}/5], Loss: {loss.item()}")

Epoch [1/5], Loss: 0.5663461685180664
Epoch [2/5], Loss: 0.5424520969390869
Epoch [3/5], Loss: 0.5068625211715698
Epoch [4/5], Loss: 0.5391815900802612
Epoch [5/5], Loss: 0.5141973495483398


In [8]:
autoencoder.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
latent_features, labels = reduce_dimensions(train_loader_auto, autoencoder.encoder, device)
latent_features = latent_features.detach()

reconstructed_images = autoencoder.decoder(latent_features.to(device))  
reconstructed_images = reconstructed_images.view(-1, 1, 28, 28)  # Reshape to [batch_size, channels, height, width]

reconstructed_dataset = CustomTensorDataset(reconstructed_images.cpu(), labels)  
reduced_train_loader_auto = DataLoader(reconstructed_dataset, batch_size=batch_size_train, shuffle=True)

# Data Partition

In [9]:
# classic
trainingset = train_loader.dataset
partitioned_data_classic = partition.balanced_dirichlet_partition(trainingset, partitions_number=4, alpha=0.5)

In [10]:
import cluster
cluster = cluster.Cluster(num_clusters=num_clusters)

targets = trainingset.targets
num_classes = len(set(targets)) 
clustered_data = cluster.apply_clustering(partitioned_data_classic, targets, num_classes)

partitioned_data_classic_clustered = clustered_data

In [11]:
classic_client_loaders = [
    DataLoader(Subset(trainingset, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_classic.values()
]

In [12]:
classic_client_loaders_clustered = [
    DataLoader(Subset(trainingset, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_classic_clustered.values()
]

In [13]:
# PCA
trainingset_pca = train_loader_reduced_pca.dataset
partitioned_data_pca = partition.balanced_dirichlet_partition(trainingset_pca, partitions_number=4, alpha=0.5)

In [14]:
import cluster
cluster = cluster.Cluster(num_clusters=num_clusters)

targets = trainingset_pca.targets
num_classes = len(set(targets)) 
clustered_data = cluster.apply_clustering(partitioned_data_pca, targets, num_classes)

partitioned_data_pca_clustered = clustered_data

In [15]:
pca_client_loaders = [
    DataLoader(Subset(trainingset_pca, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_pca.values()
]

In [16]:
pca_client_loaders_clustered = [
    DataLoader(Subset(trainingset_pca, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_pca_clustered.values()
]

In [17]:
# Autoencoder
trainingset_auto = reduced_train_loader_auto.dataset

partitioned_data_auto = partition.balanced_dirichlet_partition(trainingset_auto, partitions_number=4, alpha=0.5)

In [18]:
import cluster
cluster = cluster.Cluster(num_clusters=num_clusters)

targets = trainingset_auto.targets
num_classes = len(set(targets)) 
clustered_data = cluster.apply_clustering(partitioned_data_auto, targets, num_classes)

partitioned_data_auto_clustered = clustered_data

In [19]:
auto_client_loaders = [
    DataLoader(Subset(trainingset_auto, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_auto.values()
]

In [20]:
auto_client_loaders_clustered = [
    DataLoader(Subset(trainingset_auto, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_auto_clustered.values()
]

# Models

In [21]:
trial_model = classification_model()
trial_model_pca = classification_model()
trial_model_auto = classification_model()

global_model_pca = classification_model()
global_model_auto = classification_model()
global_model_classic = classification_model()

num_clients = 4
# classic models
local_models_classic = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]
# pca models 
local_models_pca = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]
# autoencodere models
local_model_autoencoder = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]

In [22]:
# test for errors

# Classic 

optimizer = optim.SGD(trial_model.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train(epoch, trial_model, train_loader, optimizer, log_interval, train_losses, train_counter)

c:\Users\micha\Downloads\Federated-Dimensionality-Reduction\model2.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.322914
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.309317
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.271955
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.271565
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.221788
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.217618
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.188999
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.126586
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.123816
Train Epoch: 1 [9000/60000 (15%)]	Loss: 1.976046
Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.870121
Train Epoch: 1 [11000/60000 (18%)]	Loss: 1.892674
Train Epoch: 1 [12000/60000 (20%)]	Loss: 1.821305
Train Epoch: 1 [13000/60000 (22%)]	Loss: 1.651540
Train Epoch: 1 [14000/60000 (23%)]	Loss: 1.468705
Train Epoch: 1 [15000/60000 (25%)]	Loss: 1.569610
Train Epoch: 1 [16000/60000 (27%)]	Loss: 1.482701
Train Epoch: 1 [17000/60000 (28%)]	Loss: 1.272433
Train Epoch: 1 [18000/60000 (30%)]	Loss: 1.279954
Train Epoch: 1 [19000/60000 (32%)]	Loss: 1.166096
Train Epoch: 1 [200

In [23]:
test_losses_classic_weak = []
test(trial_model,test_loader,test_losses_classic_weak)

c:\Users\micha\anaconda3\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.0980, Accuracy: 9699/10000 (97%)



In [24]:
optimizer = optim.SGD(trial_model_pca.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train(epoch, trial_model_pca, train_loader_reduced_pca, optimizer, log_interval, train_losses, train_counter)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.468736
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.251172
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.249715
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.138798
Train Epoch: 1 [4000/60000 (7%)]	Loss: 1.883769
Train Epoch: 1 [5000/60000 (8%)]	Loss: 1.760172
Train Epoch: 1 [6000/60000 (10%)]	Loss: 1.609760
Train Epoch: 1 [7000/60000 (12%)]	Loss: 1.320532
Train Epoch: 1 [8000/60000 (13%)]	Loss: 1.254012
Train Epoch: 1 [9000/60000 (15%)]	Loss: 1.103052
Train Epoch: 1 [10000/60000 (17%)]	Loss: 1.252422
Train Epoch: 1 [11000/60000 (18%)]	Loss: 0.974234
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.966534
Train Epoch: 1 [13000/60000 (22%)]	Loss: 0.976349
Train Epoch: 1 [14000/60000 (23%)]	Loss: 0.881376
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.848165
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.910968
Train Epoch: 1 [17000/60000 (28%)]	Loss: 0.822875
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.760597
Train Epoch: 1 [19000/60000 (32%)]	Loss: 0.809391
Train Epoch: 1 [200

In [25]:
test_losses_classic_pca = []
test(trial_model_pca,train_loader_reduced_pca,test_losses_classic_pca)


Test set: Avg. loss: 0.0736, Accuracy: 58625/60000 (98%)



In [26]:
optimizer = optim.SGD(trial_model_auto.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []

for epoch in range(1, n_epochs + 1):  
    train(epoch, trial_model_auto, reduced_train_loader_auto, optimizer, log_interval, train_losses, train_counter)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.318559
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.319729
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.307912
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.300609
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.288991
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.313698
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.293541
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.267741
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.287977
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.287069
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.262686
Train Epoch: 1 [11000/60000 (18%)]	Loss: 2.271168
Train Epoch: 1 [12000/60000 (20%)]	Loss: 2.278646
Train Epoch: 1 [13000/60000 (22%)]	Loss: 2.262750
Train Epoch: 1 [14000/60000 (23%)]	Loss: 2.270869
Train Epoch: 1 [15000/60000 (25%)]	Loss: 2.243819
Train Epoch: 1 [16000/60000 (27%)]	Loss: 2.229177
Train Epoch: 1 [17000/60000 (28%)]	Loss: 2.219525
Train Epoch: 1 [18000/60000 (30%)]	Loss: 2.233061
Train Epoch: 1 [19000/60000 (32%)]	Loss: 2.179585
Train Epoch: 1 [200

In [27]:
test_losses_classic_auto = []
test(trial_model_auto,reduced_train_loader_auto,test_losses_classic_auto)


Test set: Avg. loss: 0.1869, Accuracy: 56548/60000 (94%)



# federated learning loop

In [ ]:
rounds_classic = 4

for round_idx in range(rounds_classic):
    
    print(f"Round {round_idx + 1}/{rounds_classic}")

    local_weights_classic = []
    for client_idx, client_model in enumerate(local_models_classic):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, classic_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_classic.append(client_weights)
        
    print(f"after training{local_models_classic}")
    global_weights_classic = federated_averaging(local_weights_classic)
    print(f"after fedaveraging{local_models_classic}")

    distribute_global_model(global_weights_classic,local_models_classic,single=False)

    distribute_global_model(global_weights_classic,global_model_classic,single=True)
    test_losses = []
    test(global_model_classic,test_loader,test_losses)


In [ ]:


rounds_pca = 4

for round_idx in range(rounds_pca):
    
    print(f"Round {round_idx + 1}/{rounds_pca}")

    local_weights_pca = []
    for client_idx, client_model in enumerate(local_models_pca):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []


        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, pca_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_pca.append(client_weights)
        
    print(f"after training{local_models_pca}")
    global_weights_pca = federated_averaging(local_weights_pca)
    print(f"after fedaveraging{local_models_pca}")

    distribute_global_model(global_weights_pca,local_models_pca,single=False)

    distribute_global_model(global_weights_pca,global_model_pca,single=True)
    test_losses = []
    test(global_model_pca,test_loader_pca,test_losses)


    


In [ ]:
rounds_auto = 4

for round_idx in range(rounds_auto):
    print(f"Round {round_idx + 1}/{rounds_auto}")

    local_weights_auto = []
    for client_idx, client_model in enumerate(local_model_autoencoder):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)
        
        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, auto_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_auto.append(client_weights)
        
    global_weights_auto = federated_averaging(local_weights_auto)

    distribute_global_model(global_weights_auto,local_model_autoencoder,single=False)

    distribute_global_model(global_weights_auto, global_model_auto,single=True)
    test_losses = []
    test(global_model_auto,test_loader_auto,test_losses)

# Cluster

In [ ]:
rounds_classic = 4

for round_idx in range(rounds_classic):
    
    print(f"Round {round_idx + 1}/{rounds_classic}")

    local_weights_classic = []
    for client_idx, client_model in enumerate(local_models_classic):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, classic_client_loaders_clustered[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_classic.append(client_weights)
        
    print(f"after training{local_models_classic}")
    global_weights_classic = federated_averaging(local_weights_classic)
    print(f"after fedaveraging{local_models_classic}")

    distribute_global_model(global_weights_classic,local_models_classic,single=False)

    distribute_global_model(global_weights_classic,global_model_classic,single=True)
    test_losses = []
    test(global_model_classic,test_loader,test_losses)


In [ ]:


rounds_pca = 4

for round_idx in range(rounds_pca):
    
    print(f"Round {round_idx + 1}/{rounds_pca}")

    local_weights_pca = []
    for client_idx, client_model in enumerate(local_models_pca):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []


        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, pca_client_loaders_clustered[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_pca.append(client_weights)
        
    print(f"after training{local_models_pca}")
    global_weights_pca = federated_averaging(local_weights_pca)
    print(f"after fedaveraging{local_models_pca}")

    distribute_global_model(global_weights_pca,local_models_pca,single=False)

    distribute_global_model(global_weights_pca,global_model_pca,single=True)
    test_losses = []
    test(global_model_pca,test_loader_pca,test_losses)


    


In [ ]:
rounds_auto = 4

for round_idx in range(rounds_auto):
    print(f"Round {round_idx + 1}/{rounds_auto}")

    local_weights_auto = []
    for client_idx, client_model in enumerate(local_model_autoencoder):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)
        
        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, auto_client_loaders_clustered[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_auto.append(client_weights)
        
    global_weights_auto = federated_averaging(local_weights_auto)

    distribute_global_model(global_weights_auto,local_model_autoencoder,single=False)

    distribute_global_model(global_weights_auto, global_model_auto,single=True)
    test_losses = []
    test(global_model_auto,test_loader_auto,test_losses)